<a href="https://colab.research.google.com/github/MatteoBettini/Autonomous-Vehicles-Consensus-2021/blob/main/AVs_Consensus_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook initialisation

The first cell requires the user to press enter

## Install SUMO

In [ ]:
!sudo add-apt-repository ppa:sumo/stable
!sudo apt-get update
!sudo apt-get install sumo sumo-tools sumo-doc
import os
os.environ['SUMO_HOME'] = '/usr/share/sumo'
!sumo

## Clone repository

In [ ]:
!git clone https://github.com/MatteoBettini/Autonomous-Vehicles-Consensus-2021

In [ ]:
import os
os.chdir('/content/Autonomous-Vehicles-Consensus-2021')

## Install dependencies

In [ ]:
!pip install traci sumolib
!pip install ray==0.8.0

# Run simulations

In [ ]:
# To plot figures
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
import glob
import json
import subprocess
import numpy as np

from utils import PathUtils, DefaultParams
from plot_sim_results import plot_multiple_results

## Select simulation parameters

In [ ]:
# SIM PARAMS
DefaultParams.RING_RADIUS = 41.4
DefaultParams.MAX_SPEED = 30
DefaultParams.TARGET_SPEED = DefaultParams.MAX_SPEED
DefaultParams.MAX_ACCEL = 2
DefaultParams.MAX_DECEL = DefaultParams.MAX_ACCEL
DefaultParams.N_VEHICLES = 10
DefaultParams.N_BROKEN_VEHICLES = 0
DefaultParams.DURATION = 400
DefaultParams.INITIAL_HEADWAY = ((2 * np.pi * DefaultParams.RING_RADIUS) / DefaultParams.N_VEHICLES) - 5  # Vehicle length
DefaultParams.SIM_STEP = 0.1
DefaultParams.TAU = DefaultParams.SIM_STEP + 0.03

# PID HEADWAY PARAMS
DefaultParams.TIME_HEADWAY_IDM = 0.8
DefaultParams.TIME_HEADWAY = 0.6
DefaultParams.TARGET_HEADWAY = DefaultParams.TIME_HEADWAY * DefaultParams.TARGET_SPEED + 2

# INIT PARAMS
DefaultParams.BUNCHING = 0
DefaultParams.PERTURBATION = 0

# CONSENSUS PARMS
DefaultParams.V_INC = 0.2
DefaultParams.N_HOPS = -1

## Choose which controller to simulate


Config can be chosen from:

* ring_consensus_leaderless_config
* ring_idm_config
* ring_gipps_config
* ring_bando_config
* ring_follower_stopper_config
* ring_pid_headway_config

In [ ]:
config = 'ring_consensus_leaderless_config'

## Run a single simulation

In [ ]:
command = ['python',
                PathUtils.run_ring_file,
                '--exp_config', config,
                '--no_render',
                '--no_plot_outcome'
                ]

process = subprocess.Popen(command,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE)

output, error = process.communicate()

if output:
    print(output.decode())
if error:
    print(error.decode())

with open(str(PathUtils.ring_json_file), 'r') as f:
    params_dict = json.load(f)
    f.close()

plot_multiple_results(params_dict)

## Run and compare all controllers

In [ ]:
ring_configs = glob.glob(str(PathUtils.exp_configs_folder) + '/ring' + '/*.py')

for config in ring_configs:

    config = config.rsplit('/', 1)[1].rsplit('.',1)[0]

    command = ['python',
                PathUtils.run_ring_file,
                '--exp_config', config,
                '--no_render',
                '--no_plot_outcome'
                ]

    process = subprocess.Popen(command,
                                stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE)

    output, error = process.communicate()

    if output:
        print(output.decode())
    if error:
        print(error.decode())


with open(str(PathUtils.ring_json_file), 'r') as f:
    params_dict = json.load(f)
    f.close()

plot_multiple_results(params_dict)